### Importing required libraries

In [1]:
# Download beautifulsoup4 library for webscraping, if not installed
# !conda install beautifulsoup4

from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv 

### Setting `maxcolwidth` to `800` for good viewability

In [2]:
pd.set_option('max_colwidth', 800) 

### Getting the source webpage and assigining the variable `source` to it and iniatilizing the `beautifulsoup` object to `soup`


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml') 

### Initializing the `csv_writer` object and writing the name of the columns on it as the first row

In [4]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

### Scraping the page to extract the data table

In [5]:
table = soup.find('table', class_ = 'wikitable sortable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

### Closing the csv file


In [6]:
csv_file.close()

### Creating the pandas dataframe

In [7]:
df = pd.read_csv('toronto_postal_codes.csv')

### Getting the `shape` of the dataframe

In [8]:
df.shape

(103, 3)

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


### Getting the list of postal codes

In [10]:
postal_codes = df['Postcode'].values

### OpenCage GeoCoder API credentials

In [14]:
# Hidden for sharing

### Using the OpenCage GeoCoder API 

In [15]:
import json

latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for postal_code in postal_codes : 
    place_name = postal_code + " Toronto" # Formats the place name
    url = 'https://api.opencagedata.com/geocode/v1/json?q={}&key={}'.format(place_name, API_KEY) # Gets the proper url to make the API call
    obj = json.loads(requests.get(url).text) # Loads the JSON file in the form of a python dictionary
    
    results = obj['results'] # Extracts the results information out of the JSON file
    lat = results[0]['geometry']['lat'] # Extracts the latitude value
    lng = results[0]['geometry']['lng'] # Extracts the longitude value
    
    latitudes.append(lat) # Appending to the list of latitudes
    longitudes.append(lng) # Appending to the list of longitudes 

### Adding the latitude and longitude values to the dataframe

In [16]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [17]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,-33.013441,151.594204
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,Harbourfront,43.655500,-79.362600
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722300,-79.450400
4,M7A,Downtown Toronto,Queen's Park,-33.013441,151.594204


### Importing other libraries 

In [19]:
# Downloading folium, if not installed
!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

### FourSquare API Credentials

In [33]:
# Hidden for sharing

### Creating a folium map of Toronto

In [34]:
# Toronto latitude and longitude using Google search
tor_lat = 43.6532
tor_lng = -79.3832

# Creates map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lng], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Defining own function to get the name of the category

In [35]:
# Gets the name of the category

def get_category_type(row):
    try:
        categories_list = row['Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Using the FourSquare API on all neighbourhoods

In [36]:
explore_df_list = []

for i, nbd_name in enumerate(df['Neighbourhood']):  
    
    try :         
        ### Getting the data of neighbourhood
        nbd_name = df.loc[i, 'Neighbourhood']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 500 # Setting the radius as 500 metres
        LIMIT = 100 # Getting the top 100 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            explore_df_list.append([nbd_name, nbd_lat, nbd_lng] + nearby.loc[i, :].values.tolist())
    
    except Exception as e:
        pass

### Creating a dataframe for performing clustering operations

In [37]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Parkwoods,-33.013441,151.594204,Subway,Sandwich Place,-33.013180,151.592120
1,Parkwoods,-33.013441,151.594204,Woolworths,Supermarket,-33.014511,151.595918
2,Parkwoods,-33.013441,151.594204,Doubletake Cafe,Café,-33.014280,151.597680
3,Parkwoods,-33.013441,151.594204,Coles,Supermarket,-33.012820,151.595456
4,Parkwoods,-33.013441,151.594204,Michel's Patisserie,Coffee Shop,-33.012720,151.595440


### Total number of unique categories

In [38]:
len(explore_df['Venue Category'].unique())

261

### Performing one-hot encoding to analyze neighbourhoods

In [39]:
# One hot encoding
toronto_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = explore_df['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Aggregating venues by neighbourhoods

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Xinjiang Restaurant,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating a function to return the most common venues

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Creating a new dataframe to get the top 10 venues

In [42]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
1,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, Steeles East",Pharmacy,Gym,Fish Market,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Event Space,Discount Store,Ethiopian Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",Grocery Store,Pharmacy,Japanese Restaurant,Sandwich Place,Fried Chicken Joint,Beer Store,Caribbean Restaurant,Discount Store,Fast Food Restaurant,Pizza Place
4,"Alderwood, Long Branch",Pharmacy,Pub,Convenience Store,Sandwich Place,Pizza Place,Gym,Athletics & Sports,Dance Studio,Coffee Shop,Doner Restaurant


### Using K-Means clustering

In [43]:
# Set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

### Creating a grand dataframe containing the necessary data

In [44]:
toronto_merged = df
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,-33.013441,151.594204,1,Café,Supermarket,Fast Food Restaurant,Harbor / Marina,Sandwich Place,Waterfront,Fried Chicken Joint,Coffee Shop,Electronics Store,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.727600,-79.314800,1,Hockey Arena,Coffee Shop,Intersection,French Restaurant,Portuguese Restaurant,Park,Pizza Place,Farmers Market,Event Space,Ethiopian Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.655500,-79.362600,1,Coffee Shop,Restaurant,Breakfast Spot,Mexican Restaurant,Italian Restaurant,Thai Restaurant,Pub,Bakery,Thrift / Vintage Store,Food Truck
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722300,-79.450400,1,Clothing Store,Coffee Shop,Cosmetics Shop,Fast Food Restaurant,Electronics Store,Bakery,Sandwich Place,Toy / Game Store,Food Court,Men's Store
4,M7A,Downtown Toronto,Queen's Park,-33.013441,151.594204,1,Café,Pharmacy,Supermarket,Bank,Coffee Shop,Sandwich Place,Skating Rink,Fast Food Restaurant,Fried Chicken Joint,Park


### Visualizing the clusters

In [45]:
# Create map
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

### Cluster 0

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,Berczy Park,0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
30,Downtown Toronto,"Adelaide, King, Richmond",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
33,North York,"Fairview, Henry Farm, Oriole",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
34,North York,"Northwood Park, York University",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
36,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
43,West Toronto,"Brockton, Exhibition Place, Parkdale Village",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
59,North York,Willowdale South,0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
63,York,"The Junction North, Runnymede",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
69,West Toronto,"High Park, The Junction South",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
81,West Toronto,"Runnymede, Swansea",0,Women's Store,Home Service,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


### Cluster 1

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,1,Café,Supermarket,Fast Food Restaurant,Harbor / Marina,Sandwich Place,Waterfront,Fried Chicken Joint,Coffee Shop,Electronics Store,Dumpling Restaurant
1,North York,Victoria Village,1,Hockey Arena,Coffee Shop,Intersection,French Restaurant,Portuguese Restaurant,Park,Pizza Place,Farmers Market,Event Space,Ethiopian Restaurant
2,Downtown Toronto,Harbourfront,1,Coffee Shop,Restaurant,Breakfast Spot,Mexican Restaurant,Italian Restaurant,Thai Restaurant,Pub,Bakery,Thrift / Vintage Store,Food Truck
3,North York,"Lawrence Heights, Lawrence Manor",1,Clothing Store,Coffee Shop,Cosmetics Shop,Fast Food Restaurant,Electronics Store,Bakery,Sandwich Place,Toy / Game Store,Food Court,Men's Store
4,Downtown Toronto,Queen's Park,1,Café,Pharmacy,Supermarket,Bank,Coffee Shop,Sandwich Place,Skating Rink,Fast Food Restaurant,Fried Chicken Joint,Park
5,Queen's Park,Queen's Park,1,Café,Pharmacy,Supermarket,Bank,Coffee Shop,Sandwich Place,Skating Rink,Fast Food Restaurant,Fried Chicken Joint,Park
6,Scarborough,"Rouge, Malvern",1,Café,Supermarket,Fast Food Restaurant,Harbor / Marina,Sandwich Place,Waterfront,Fried Chicken Joint,Coffee Shop,Electronics Store,Dumpling Restaurant
8,East York,"Woodbine Gardens, Parkview Hill",1,Fast Food Restaurant,Pizza Place,Pharmacy,Gym / Fitness Center,Gastropub,Intersection,Bank,Creperie,Cuban Restaurant,Field
9,Downtown Toronto,"Ryerson, Garden District",1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Bakery,Fast Food Restaurant,Bookstore
10,North York,Glencairn,1,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Rental Car Location,Latin American Restaurant,Mediterranean Restaurant,Italian Restaurant,Gas Station,Fish Market,Grocery Store


### Cluster 2

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,"Silver Hills, York Mills",2,Pool,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space
46,North York,Downsview West,2,Pool,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space


### Cluster 3

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,Don Mills North,3,Park,Basketball Court,Pool,Yoga Studio,Empanada Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space
27,North York,Hillcrest Village,3,Residential Building (Apartment / Condo),Park,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
32,Scarborough,Scarborough Village,3,Grocery Store,Yoga Studio,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Discount Store
35,East York,East Toronto,3,Park,Massage Studio,Coffee Shop,Convenience Store,Harbor / Marina,Diner,Field,Fast Food Restaurant,Farmers Market,Event Space
39,North York,Bayview Village,3,Flower Shop,Trail,Park,Gas Station,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
61,Central Toronto,Lawrence Park,3,Photography Studio,Park,Discount Store,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Empanada Restaurant
64,York,Weston,3,Convenience Store,Park,Dog Run,Fish Market,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant
66,North York,York Mills West,3,Convenience Store,Park,Bank,Empanada Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space
68,Central Toronto,"Forest Hill North, Forest Hill West",3,Trail,Park,Food Service,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
73,Central Toronto,North Toronto West,3,Playground,Gym Pool,Park,Garden,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


### Cluster 4

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,Humber Summit,4,Auto Garage,Yoga Studio,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Dog Run
71,Scarborough,"Maryvale, Wexford",4,Convenience Store,Auto Garage,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Dog Run
